In [35]:
from z3 import *
import math

def print_poly_from_model(m):
  # Try to extract A and B from the model by variable names
  # Find all variables in the model, group by 'a' and 'b'
  A_vars = []
  B_vars = []
  for d in m.decls():
    name = d.name()
    if name.startswith('a'):
      idx = int(name[1:])
      A_vars.append((idx, m[d].as_long()))
    elif name.startswith('b'):
      idx = int(name[1:])
      B_vars.append((idx, m[d].as_long()))
  # Sort by index
  A_vars.sort()
  B_vars.sort()
  def poly_to_str(coeffs):
    terms = []
    for i, coef in coeffs:
      if coef == 0:
        continue
      if i == 0:
        terms.append(f"{coef}")
      elif i == 1:
        terms.append(f"{coef}*x" if coef != 1 else "x")
      else:
        terms.append(f"{coef}*x^{i}" if coef != 1 else f"x^{i}")
    return " + ".join(reversed(terms)) if terms else "0"
  polyA = poly_to_str(A_vars)
  polyB = poly_to_str(B_vars)
  print(f"({polyA})({polyB})")

def is_reducible_Zn(px, n):
    deg_f = len(px) - 1  # derajat polinomial f(x)

    # Coba semua pasangan derajat A dan B yang jumlahnya = deg_f
    for deg_A in range(1, math.floor(deg_f/2)):  # minimal derajat 1
        deg_B = deg_f - deg_A

        A = [Int(f"a{i}") for i in range(deg_A + 1)]
        B = [Int(f"b{i}") for i in range(deg_B + 1)]

        AB = [0] * (deg_f + 1)  # hasil perkalian A*B

        for i in range(deg_A + 1):
            for j in range(deg_B + 1):
                AB[i + j] += A[i] * B[j]

        s = Solver()
  
        # Semua koefisien di Z_n
        for coef in A + B:
            s.add(coef >= 0, coef < n)
    
        # Cocokkan koefisien hasil A*B dengan px mod n
        for i in range(deg_f + 1):
            s.add(AB[i] % n == px[i])

        if s.check() == sat:
            print(f"Reducible in Z_{n} as deg(A)={deg_A}, deg(B)={deg_B}")
            while s.check() == sat:
              m = s.model()
              print_poly_from_model(m)
              # Tambahkan constraint agar solusi berikutnya berbeda
              s.add(Or([a != m[a] for a in A + B]))
            return 

    print(f"Irreducible in Z_{n}")
    return 

# Contoh:
px = [1, 4, 1, 3, 2]  # derajat 4
n = 5
is_reducible_Zn(px, n)


Reducible in Z_5 as deg(A)=1, deg(B)=3
(4*x + 3)(3*x^3 + x^2 + 2*x + 2)
(3*x + 1)(4*x^3 + 3*x^2 + x + 1)
(x + 2)(2*x^3 + 4*x^2 + 3*x + 3)
(2*x + 4)(x^3 + 2*x^2 + 4*x + 4)
